<a href="https://colab.research.google.com/github/Neulvo/TIL/blob/master/10_Prac_2_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10장 문서 분류 (Document Classification)

#11-1 나이브 베이즈 분류(Naive Bayes Classifier)

##1.1 직접구현

### Naive Bayes Classifier

In [1]:
training_set = [['me free lottery', 1],
 ['free get free you', 1],
 ['you free scholarship', 0],
 ['free to contact me', 0],
 ['you won award', 0],
 ['you ticket lottery', 1]]

### 토큰 빈도수 및 문서별 토큰수 계산 (확률 계산을 위한 준비)

![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/98f086c560aa2f66650060277dda4f90e54e30c0)

In [2]:
training_set[0]

['me free lottery', 1]

In [5]:
from collections import defaultdict

# 범주에 속하는 토큰수 세기 1(스팸), 0(정상))
doccnt0 = 0
doccnt1 = 0

# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])

for doc, label in training_set:
  words = doc.split()
  for word in words:
    wordfreq[word][label] += 1

for key, (cnt0, cnt1) in wordfreq.items():
  doccnt0 += cnt0
  doccnt1 += cnt1

print('doccnt0 : {}'.format(doccnt0))
print('doccnt1 : {}'.format(doccnt1))

doccnt0 : 10
doccnt1 : 10


In [6]:
doccnt0

10

In [7]:
doccnt1

10

### Training : 토큰별 조건부 확률 계산 

In [8]:
k= 0.5

wordprobs = defaultdict(lambda : [0,0])
for key, (cnt0, cnt1) in wordfreq.items():
  wordprobs[key][0] = (cnt0+k)/(2*k+doccnt0)
  wordprobs[key][1] = (cnt1+k)/(2*k+doccnt1)
wordprobs

defaultdict(<function __main__.<lambda>>,
            {'award': [0.13636363636363635, 0.045454545454545456],
             'contact': [0.13636363636363635, 0.045454545454545456],
             'free': [0.22727272727272727, 0.3181818181818182],
             'get': [0.045454545454545456, 0.13636363636363635],
             'lottery': [0.045454545454545456, 0.22727272727272727],
             'me': [0.13636363636363635, 0.13636363636363635],
             'scholarship': [0.13636363636363635, 0.045454545454545456],
             'ticket': [0.045454545454545456, 0.13636363636363635],
             'to': [0.13636363636363635, 0.045454545454545456],
             'won': [0.13636363636363635, 0.045454545454545456],
             'you': [0.22727272727272727, 0.22727272727272727]})

### Classify : 신규 텍스트가 주어졌을 때 확률 계산

>



In [12]:
import math

doc= 'free lottery '
tokens = doc.split()

log_prob1 = log_prob0 = 0.0

for word, (prob0, prob1) in wordprobs.items():
  if word in tokens:
    log_prob0 += math.log(prob0)
    log_prob1 += math.log(prob1)

log_prob0 += math.log(doccnt0/(doccnt0 + doccnt1))
log_prob1 += math.log(doccnt1/(doccnt0 + doccnt1))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print(prob0)
print(prob1)

print('정상 확률 : {}'.format(prob0 /(prob0+prob1)*100))
print('스팸 확률 : {}'.format(prob1/(prob0+prob1)*100))

0.00516528925619835
0.03615702479338842
정상 확률 : 12.500000000000009
스팸 확률 : 87.49999999999999


In [13]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names)
print(twenty_train.data[0])
print(twenty_train.target[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [14]:
from sklearn.pipeline import Pipeline 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [15]:
MultinomialNB?

In [16]:
text_clf

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [17]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True) 
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted==twenty_test.target)

0.7738980350504514

In [19]:
from sklearn.model_selection import GridSearchCV 
parameters_clf = {'vect__ngram_range': [(1,1), (1,2)],
                  'tfidf__use_idf': (True, False),
                  'clf__alpha' : (1, 0.1, 0.01, 0.001, 0.0001)
                  }
gs_clf = GridSearchCV(text_clf, parameters_clf, cv=2, n_jobs=-1, verbose=2)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
print('Best Score : {}'.format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(list(best_parameters.keys())):
  print("\t{} : {}".format(param_name, best_parameters[param_name]))                

Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.7min finished


Best Score : 0.893759943432915
	clf : MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
	clf__alpha : 0.01
	clf__class_prior : None
	clf__fit_prior : True
	memory : None
	steps : [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))]
	tfidf : TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
	tfidf__norm : l2
	tfidf__smooth_idf : True
	tfidf__sublinear_tf : False
	tfidf__use_idf : True
	vect : CountVectorizer(analyzer='word'

In [20]:
predicted = gs_clf.best_estimator_.predict(twenty_test.data)
np.mean(predicted== twenty_test.target)

0.8352363250132767

1.3 sklearn 활용 (한글 뉴스 분류)

In [21]:
!wget https://github.com/kyungsoo-fininsight/mulcam_b/raw/master/data/2019news_1000.csv

--2020-08-02 23:47:40--  https://github.com/kyungsoo-fininsight/mulcam_b/raw/master/data/2019news_1000.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kyungsoo-fininsight/mulcam_b/master/data/2019news_1000.csv [following]
--2020-08-02 23:47:41--  https://raw.githubusercontent.com/kyungsoo-fininsight/mulcam_b/master/data/2019news_1000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11125708 (11M) [text/plain]
Saving to: ‘2019news_1000.csv’

2019news_1000.csv   100%[===================>]  10.61M  20.1MB/s    in 0.5s    

2020-08-02 23:47:43 (20.1 MB/s) - ‘2019news_1000.csv’ saved [11125708/1112

In [22]:
%%time
import pandas as pd
df = pd.read_csv("./2019news_1000.csv")

CPU times: user 148 ms, sys: 14 ms, total: 162 ms
Wall time: 163 ms


In [23]:
df.head(10)

,url,category1,category2,date,title,media,content
0,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,모바일,2019-05-02,"인권단체 중국, 신장위구르 소수민족 감시용 모바일앱 가동",연합뉴스,"HRW ""개인 정보 수집·보고서 작성·조사 활동에 앱 활용"" ""36가지 감시유형…뒷..."
1,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,모바일,2019-05-09,"카카오, 1분기 매출 7000억 넘어서…8분기째 최고치 경신(종합)",뉴시스,영업익 166.0%↑·순이익 19.
2,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,과학 일반,2019-12-11,"테라젠이텍스, '유전체 정보 관리 시스템' 특허 취득",연합뉴스,(서울=연합뉴스) 김잔디 기자 = 테라젠이텍스는 유전체 분석 정보 관리 시스템에 관...
3,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,컴퓨터,2019-10-14,"두나무-삼성증권-딥서치, 비상장 주식 통합 거래 지원 플랫폼 출범",디지털데일리,'증권플러스 비상장' 서비스를 설명중인 두나무 이성현 핀테크사업실장 [디지털데일리 ...
4,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,통신/뉴미디어,2019-10-24,"과기정통부, 태풍 미탁 피해, 특별재난지역 전파사용료 6개월간 전액감면",전자신문,과기정통부 로고 과학기술정보통신부는 18호 태풍 '미탁'으로 인해 특별재난지역으로 ...
5,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,게임/리뷰,2019-04-09,"모바일로 재현된 ‘더 킹 오브 파이터즈’...'KOF 올스타', 한국 사전등록 실시",OSEN,[OSEN=고용준 기자] 강렬한 액션과 호쾌한 타격감으로 세계적인 인기를 끌었던 '...
6,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,통신/뉴미디어,2019-10-24,양자암호통신 상용화 위해 ETRI-통신 3사 표준화 착수,파이낸셜뉴스,ETRI 네트워크연구본부에서 전송시스템기술 개발을 수행하고 있는 모습. ETRI 제...
7,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,인터넷/SNS,2019-10-12,"용인시, 대학?연구기관 첨단장비 대여제 중기서 큰 호응",전자신문,용인시청 전경. 용인시는 대학이나 연구기관이 보유한 고가 첨단장비를 저렴한 비용으로...
8,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,인터넷/SNS,2019-08-07,"전문연구요원제, 정부 생각은?.국방부 “양측 고려 시기 규모 결정”",이데일리,"국방부 절충과 균형안 제시..철회는 아냐 중기부, 산업부, 과기부는 상대적으로 산업..."
9,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,인터넷/SNS,2019-04-19,"에픽게임즈스토어, SF 액션 RPG ‘트랜지스터’ 무료 배포",매일경제,에픽게임즈코리아(대표 박성철)는 에픽게임즈스토어를 통해 SF 액션 RPG ‘트랜지스...


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['content'], df['category1'],random_state=0)
print(X_train)
print(y_train)
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf',MultinomialNB(alpha=0.5))])
text_clf = text_clf.fit(X_train, y_train)

4154    봉산문화회관에서 10월 15일부터 20일까지 홍창진 개인전 '오직, 회화 - 우리도...
4820    에이유커머스가 전개하는 슈즈 브랜드 락피쉬가 일러스트 작가 울랄라 1537과 함께 ...
1202    【서울=뉴시스】로버트 에이브럼스(오른쪽 네번째) 한미 연합사령관 겸 주한미군 사령관...
3759    한샘 라이너스 중문 / 사진제공=한샘 [서울경제] 한샘은 이달 말까지 ‘원데이 리모...
622     홍보 모델들이 순천향대학교 서울병원 병실에 구축된 에어맵 플랫폼을 소개하고 있다. ...
                              ...                        
4931    출처: 제너시스BBQ BBQ '뱀파이어 치킨'이 출시돼 화제를 모으고 있다. 1일 ...
3264    제20회 서울국제금융포럼이 파이낸셜뉴스와 오크트리 주최로 서울 소공로 웨스틴 조선호...
1653    3차 美北정상회담 가능성 재확인은 소기의 성과 미국산 구미 구매·방위비 분담금 인상...
2607    전남지역 조선업체[전남도 제공] (무안=연합뉴스) 여운창 기자 = 전남지역 특성화고...
2732    연말연시 모금 2년 연속 목표 밑돌 듯 경남모금회, 희망 2019 나눔 캠페인 발대...
Name: content, Length: 3750, dtype: object
4154    생활/문화
4820    생활/문화
1202       정치
3759       경제
622     IT/과학
        ...  
4931    생활/문화
3264       경제
1653       정치
2607       사회
2732       사회
Name: category1, Length: 3750, dtype: object


In [26]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted ==  y_test)

0.6832

In [ ]:
from sklearn.model_selection import GridSearchCV 
parameters_clf = {'vect__ngram_range':[(1,1), (1,2), (1,3), (1,4)],
                  'tfidf__use_idf':(True, False),
                  'clf__alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}
gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)

In [29]:
best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(list(best_parameters.keys())):
  print('\t{}:{}'.format(param_name, best_parameters[param_name]))


	clf:MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
	clf__alpha:0.01
	clf__class_prior:None
	clf__fit_prior:True
	memory:None
	steps:[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))]
	tfidf:TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
	tfidf__norm:l2
	tfidf__smooth_idf:True
	tfidf__sublinear_tf:False
	tfidf__use_idf:True
	vect:CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                

In [30]:
predicted=gs_clf.best_estimator_.predict(X_test)
np.mean(predicted == y_test)

0.7408